Analisando a página da Drogaria Minas Mais, percebo que ela é uma página dinâmica (tem o botão de "Carregar mais produtos")
Outro fator que também chamou atenção: ao desligar o Javascript da página, ela não exibiu nenhum produto (sequer conseguiu carregar), exibindo uma dependenciaforte do JS
Assim, vou tentar uma abordagem para um HTML dinâmico 



No site há 6118 produtos, sendo que na primeira página há 15 produtos, na segunda há outros 15 produtos e assim por diante 
Temos um total de 408 páginas
Entretanto na página 51 já não encontramos mais produtos (Acredito que o acesso esteja limitado pelo parâmetro Page, como previa o arquivo de treinamento interno do Notion) 


Vou tentar fazer o processo de filtrar por marcas
A marca com mais produtos é a HYPERA com 371 produtos
O layout do site apresenta 15 produtos por página. Como podemos acessar 50 páginas, temos acesso a 750 (50x15) produtos no máximo
Assim filtrar por marca realmente será uma estratégia válida

<label class="vtex-checkbox__label w-100 c-on-base pointer" for="brand-hypera">HYPERA</label>
<label class="vtex-checkbox__label w-100 c-on-base pointer" for="brand-ache-rx">ACHE RX</label>

Cliquei em inspect em cima das marcas e achei isso no código fonte.
Dar um jeito de pegar todas as marcas com essa estrutura

In [451]:
from bs4 import BeautifulSoup
import requests
import json
import re
import pandas as pd
import math

from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait

In [452]:
def pegar_html(link):  
    req=requests.get(link)
    html=BeautifulSoup(req.text,"html.parser")
    return html



In [453]:
def achar_marcas(html):
    
    marcas=html.find_all("label", attrs={"class":"vtex-checkbox__label w-100 c-on-base pointer"})
    strmarcas=[]
    POP=[]
    REMOVER=[]

    #transformar ps códigos <label ...  em strings
    for elem in marcas:
        strmarcas.append(str(elem))
    

    #achar os índices das labels(em string) que possuem for="category-2...", que não representam marcas
    for i in range(len(strmarcas)):
        if "category-2" in strmarcas[i]:
            POP.append(i)

    #Achar a correlação entre as labels(em string) e as labels normais. Saber qual label normal deve ser removida
    for indice in POP:
        REMOVER.append(marcas[indice])

    for indesejado in REMOVER:
        marcas.remove(indesejado)

    #pegando somente o nome das marcas (retirando toda a estrutura do label)
    TEXTO=[]
    for label in marcas:
        TEXTO.append(label.string)

    #analisando o sorted(TEXTO), percebi que diversos elementos da lista eram duplicados

    #Tirando os nomes duplicados
    TEXTO_S_DUPL=[]
    for elem in TEXTO:
        if elem not in TEXTO_S_DUPL:
            TEXTO_S_DUPL.append(elem) 


    #formatando o nome da marca
    MARCA_PARA_LINK=[]
    for elem in TEXTO_S_DUPL:
        elem=elem.replace(" ","-")
        elem=elem.replace("/","-")
        elem=elem.lower()
        MARCA_PARA_LINK.append(elem)

    #print(strmarcas)
    #print(POP)
    #print(REMOVER)
    return MARCA_PARA_LINK




In [454]:
def link (MARCA_LINK):
    LINK=[]
    for i in MARCA_LINK:
        link=f"https://www.drogariasminasmais.com.br/medicamentos/{i}?initialMap=c&initialQuery=medicamentos&map=category-1,brand"
        link+="&page="
        LINK.append(link)
    return(LINK)


Analisando a página da Hypera, pelo ThunderClient, posso perceber que todo o dicionário de interesse está na variável __STATE__ 
Mas ele só devolveu informações da primeira página. 

Acredito que o jeito mais facil será fazer requests das páginas dos jsons (v1 ao inspecionar a página) e achar como os links evoluem. A ideia é fazer requests em cima desses links e extrair as informações

In [455]:
def qtd_produtos(json,marca):
    nome='$ROOT_QUERY.facets({"behavior":"Static","categoryTreeBehavior":"default","hideUnavailableItems":true,"initialAttributes":"c","query":"medicamentos/ache-rx","selectedFacets":[{"key":"category-1","value":"medicamentos"},{"key":"brand","value":"ache-rx"}]})@context({"provider":"vtex.search-graphql"})@runtimeMeta({"hash":"d0cbc665c0364a2c77575f998051aa1a9f8a04a60174d78356b18b0406fe48b6"}).facets.3.values({}).'
    for key,values in json.items():
        if key.startswith(nome):
            

            #alterar a formatação da marca
            marca=marca.replace("-"," ")
            marca=marca.upper()

            for sub_key,sub_value in values.items():
                if sub_key=='quantity':
                    quantidade=sub_value
                    #print(sub_value)
                    #print(marca)
                if sub_key=='name' and sub_value==marca:
                    return quantidade


In [456]:
def get_json(url):
        requisicao = requests.get(url)
        soup = BeautifulSoup(requisicao.text, 'html.parser')

        string = soup.find_all('script')[-34].text #Converter para arquivo de texto (retira as tags)

        json_file = string.split("__STATE__ = ", 1) #1 é o indice (lista é criada com o split)
        
        return json.loads(json_file[1])



In [457]:
def get_ean(json):
    EAN=[]
    for key,value in json.items():
        if key.startswith('Product:sp-'):
            for sub_key,sub_value in value.items():
                if sub_key == "ean":
                    EAN.append(sub_value)

    return(EAN)
        





In [458]:
def get_name(json):
    NOMES=[]
    for key,value in json.items():
        if key.startswith('Product:sp-'):
            for sub_key,sub_value in value.items():
                if sub_key=='productName':
                    sub_value=sub_value.split(" -")[0]
                    NOMES.append(sub_value)
    return NOMES


In [459]:
def get_preço_c_desconto(json):
    VENDA=[]
    for key,value in json.items():
        if '$Product:sp-' and '.priceRange.sellingPrice' in key:
            for sub_key,sub_value in value.items():
                if sub_key == 'highPrice':
                    VENDA.append(sub_value)
    return VENDA




In [460]:
def get_preço_s_desconto(json):
    CHEIO=[]
    for key,value in json.items():
        if '$Product:sp-' and '.priceRange.listPrice' in key:
            for sub_key,sub_value in value.items():
                if sub_key == 'highPrice':
                    CHEIO.append(sub_value)
    return CHEIO


    

In [461]:
def desconto(A,B):
    DESCONTO=[]
    for i in range(len(A)):
        x= B[i] - A[i]
        if x<0:
            x=(-1)*x
            x=round(x,2)
            DESCONTO.append(x)
        else:
            x=round(x,2)
            DESCONTO.append(x)
    return DESCONTO

     

In [462]:
colunas=['EAN','Marca','Nome','Preço com desconto','Preço sem desconto','Desconto']
df=pd.DataFrame(columns=colunas)


url='https://www.drogariasminasmais.com.br/medicamentos'
html=pegar_html(url)
marcas=achar_marcas(html)
LINKS=link(marcas)
""" print(LINKS[0])
json=get_json(LINKS[0])
ean=get_ean(json)
nomes_produtos=get_name(json)
p_des=get_preço_c_desconto(json)
p_s_des=get_preço_s_desconto(json)
desconto=desconto(p_s_des,p_des)

df['EAN']=ean
df['Marca']=marcas[0]
df['Nome']=nomes_produtos
df['Preço com desconto']=p_des
df['Preço sem desconto']=p_s_des
df['Desconto']=desconto
df """


https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1
https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=2
https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=3
https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=4


In [463]:
for i in range(1,5):
    print (i)

1
2
3
4


In [464]:
x=14.3
x = math.ceil(x)
x

15

In [465]:
""" def get_informations(url):
    file = get_json(url)

    df = pd.DataFrame(columns=["Name","Product ID", "Brand"])


    for chave, detalhes_produto in file.items():

        if chave.startswith('Product:sp-') and chave.split('-')[1].isdigit() and int(chave.split('-')[1]) != 0:

            name = detalhes_produto["productName"]
            product_id = detalhes_produto["productId"]
            brand = detalhes_produto["brand"]
            cache_id = detalhes_produto.get("cacheId", "") """

""" def get_ean(url):
    file = get_json(url)

    ean_values = []

    for chave, detalhes_produto in file.items():
        if chave.startswith('Product:sp-'):
            
            for sub_chave, valor in detalhes_produto.items():

                if 'ean' in sub_chave:
                    ean_values.append(valor)
                    break

    df2 = pd.DataFrame({"EAN": ean_values})

    coluna_ean = df2['EAN']

    informations = get_informations(url)

    informations['Ean'] = coluna_ean """

' def get_ean(url):\n    file = get_json(url)\n\n    ean_values = []\n\n    for chave, detalhes_produto in file.items():\n        if chave.startswith(\'Product:sp-\'):\n            \n            for sub_chave, valor in detalhes_produto.items():\n\n                if \'ean\' in sub_chave:\n                    ean_values.append(valor)\n                    break\n\n    df2 = pd.DataFrame({"EAN": ean_values})\n\n    coluna_ean = df2[\'EAN\']\n\n    informations = get_informations(url)\n\n    informations[\'Ean\'] = coluna_ean '